In [ ]:
import numpy as np
import math
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as p

# Part III
### Learning

Vi finder først en måde/metode til at estimere parametrene fra noget observeret data. Dette bruger vi også senere til EM algoritmen. Vi omskriver forward_sim uden faste variabel deklerationer. 

In [ ]:
def forward_sim(n, T, alpha, gamma, beta, lmda): 
    # Pre-defined (transition) probabilities
    Gamma = [[1-gamma, 0, gamma], [0, 1-gamma, gamma], [beta*0.5, beta*0.5, 1-beta]]
    
    # Initialize arrays
    C = np.zeros(T, dtype=int)
    Z = np.zeros((T, n), dtype=int)
    X = np.zeros((T, n), dtype=int)
    
    Z_prob = [[alpha, 1-alpha], [1-alpha, alpha], [0.5, 0.5]]

    # Initial state
    C[0] = 2

    # Initialize X and Z for t=0
    for i in range(0, n):
            Z[0, i] = random.choices([0, 1], Z_prob[C[0]])[0]
            X[0, i] = np.random.poisson(lam = lmda[Z[0, i]])

    # Simulate the proces for the remaining t=1 to t=T timesteps
    for t in range(1, T):
        C[t] = random.choices([0, 1, 2], Gamma[C[t-1]])[0]
        for i in range(0, n):
            Z[t, i] = random.choices([0, 1], Z_prob[C[t]])[0]
            X[t, i] = np.random.poisson(lam = lmda[Z[t, i]])

    return C, Z, X

Efter at have en måde til at estimere nogen parametre, kan vi antage at det er de rigtige parametre, og bruge dette til at infer nye c og z. 

Initialization

In [ ]:
alpha=0.9 # Probability parameter alpha in (0.5, 1) for P(Z|C)
beta=0.2  # Transition probability parameter for moving to and from parallel processing
gamma=0.1  # Transition probability parameter between states
lmda=(1, 5)  # Rate parameters for the Poisson distribution modeling spike counts under each attention state.
Gamma = np.array([[1-gamma, 0, gamma], [0, 1-gamma, gamma], [beta*0.5, beta*0.5, 1-beta]])
T = 100  # Time period
n = 10   # Number of neurons

C, Z, X = forward_sim(n, T,alpha=0.9, gamma=0.1, beta=0.2, lmda=(1, 5))

In [ ]:
def PX_given_Z(x, z, lmda=(1, 5)):
    return (np.exp(-1 * lmda[z]) * ((lmda[z]**x)/math.factorial(x)))

Z_marg = (1-alpha, alpha) #We assume that P(Z=0) = 0.5 due to the distribution of Z | C, and due to an assumption that P(C=1) = P(C=0)

#We start by calculating P(X) for every observed X and save them in a table. It doesn't make sense to compute P(X) for unobserved values of X
X_marg = np.zeros(np.max(X)+1)
#We calculate P(X) using the formula: P(X=x) = P(X=x|Z=0)P(Z=0) + P(X=x|Z=1)P(Z=1)
#This works because Z ony takes values in {0,1}. We simply obtain the joint distribution P(X,Z) and sum out Z
for i in range(len(X_marg)):
    X_marg[i] = PX_given_Z(i, 0)*Z_marg[0] + PX_given_Z(i, 1)*Z_marg[1] 
#X_marg[i] holds the answer for the query P(X=i), Likewise Z_marg[i] holds the answer for the query p(Z=i)


#For every observed X=x, we calculate the conditional probaility P(Z=0|X=x)
#We use Bayes theorem to do this. 
Z0_cond = np.zeros_like(X, dtype=float) 

for i in range(Z0_cond.shape[0]): 
    for j in range(Z0_cond.shape[1]): 
        Z0_cond[i,j] = (PX_given_Z(X[i,j], 0)*Z_marg[0])/(X_marg[X[i,j]]) #P(Z|X)

def PZ_given_C(c, z, alpha=0.9):
    Z_prob = np.array([[alpha, 1-alpha], [1-alpha, alpha], [0.5, 0.5]])
    return Z_prob[c, z]


################## Forward Algorithm ##################
def calc_emission_proba(t: int, c: int) -> float:
    """
    Calculates the emission probabilities at timestep t
    The probabilities given by P(X_1,t...X_n,t|C_t)
    P(Z|C) by function PZ_given_C(c, z)
    P(X|Z) Poisson distribution given by PX_given_Z(x, z)
    """
    emission_proba = 0.
    for i in range(n):
        emission_proba += np.log(PZ_given_C(c, 0)*PX_given_Z(X[t, i], 0) + PZ_given_C(c, 1)*PX_given_Z(X[t, i], 1))
    emission_proba = np.exp(emission_proba)
    return emission_proba

def forward():
    """
    Iteratively updates the Alpha_t vectors in the Alpha matrix.
    The Alpha matrix represents the P(C|X) distribution for each timestep T as an T x 3 matrix.
    """
    # Initialization of the forward probabilities matrix, Alpha.
    Alpha = np.zeros((T, 3), dtype=float)  # T x 3 matrix P(C|X)
    Alpha[0, 2] = 1.  # Assumed state C_0=2 becomes our Alpha_0 vector (Alpha_0=[0, 0, 1])
    for t in range(1, T):
        for c in range(3):
            c_prev = 0. # For calculating SUM_c_t-1 P(C_t|c_t-1)*Alpha_t-1(c_t-1)
            for c_ in range(3):
                c_prev += Gamma[c_, c]*Alpha[t-1, c_]
            Alpha[t, c] = calc_emission_proba(t, c)*c_prev # Alpha_t = P(X_1,t,...,X_n,t|C_t) * SUM_c_t-1 P(C_t|c_t-1)*Alpha_t-1(c_t-1)
        row_sum = np.sum(Alpha[t], axis=0) # Normalizing sum for each row
        Alpha[t] = Alpha[t]/row_sum # Row-wise normalization for the CPD, normalizing the unnormalized Gibbs measure of each row.
    return Alpha

In [ ]:
Alpha = forward() # Store P(C|X) in an T x 3 matrix.
Alpha[30:50] # P(C|X) of the 19 timesteps t=30 to t=49
C[30:50] # Simulated values of the hidden layer C from t=30 to t=49

Prøver at lave en generel metode til at cementere værdierne til at være ny data.

In [ ]:
T = 100  # Time period
n = 10   # Number of neurons



In [ ]:
print(Z.shape)

In [ ]:
def cement_data(inferd_Z, inferd_C):
    Z = np.zeros((T,n),dtype=int)
    C = np.zeros(T,dtype=int)
    print(inferd_Z[0][0])
    for t in range(T):
        for i in range(n):
            Z[t,i] = np.argmax(inferd_Z[t][i]) #giver det index hvor der er størst værdi i vores confidence matrix
    
    for t in range(T):
        C[t] = np.argmax(inferd_C[t])
            
    return Z, C

print(Z0_cond.shape)
print(Alpha.shape)
#cement_data(Z0_cond,Alpha)[1]